# Orlando Criminal Activities

##### Presented by: Clark Jones, Umapathy Beema, Richard Morand <br> Dr. Spencer Lyon - Spring 2024 


Objective: <br>
Each residential area are formed by many factors that constributes to its presence. From social events like volunteering bringing a positive status in the communities to criminal activities, which leads to negative insights on neighborhoods. These different actions could make or break the community. <br>


For this experiement, we will be exploring the criminal acts in Orlando, and create different models to showcase the level of danger across the city.<br>

__Tools__ <br>
Collaboration: VSCode, Github, Discord <br>
Languages: Python, Julia <br>
Packages/Libraries: Agents, OpenStreetMap, Folium, etc.

In [ ]:
# using Pkg
# Pkg.update()

#import/use packages and libraries here
import Pkg
# Pkg.add(["OpenStreetMapX", "CSV", "DataFrames", "Geodesy", "Conda", "PyCall" ], io=devnull)

# using Agents
using OpenStreetMapX, Geodesy
using CSV, DataFrames, Conda, PyCall

Conda.runconda(`install folium -c conda-forge -y`) # Folium Map
# Conda.runconde(`conda update -n base -c conda-forge conda`) ## Update Conda version
foli = pyimport("folium")


In [3]:
# load and analyze data
data = CSV.read("Data/OPD_Officer-Involved_Shootings_20241125.csv", DataFrame)

Row,Case #,Date,Address,Incident Type,Number of Officers Involved,Officer Name,Officer Race,Ethnicity,Officer Gender,Officer Age,Officer Tenure,Number of Suspects Involved,Suspect Name,Suspect Race,Suspect Gender,Suspect Age,Suspect's Weapon,Suspect Hit,Fatal,Officer Hit,Fatal (Officer),Review Letter,Georeferenced Column
,String15,String15,String,String,Int64?,String?,String?,String31?,String31,String,String,Int64,String,String31,String7,String15,String31,String15,String15,String,String,String?,String
1,22-174307,05/18/2022,Mercy Drive and W.D. Judge Drive,Vehicle Stop,2,Marsy's Law Exempt,White/White,missing,"M,F",Marsy's Law Exempt,Marsy's Law Exempt,1,Carlos Delano Dafill Roberts Jr.,Black,M,29,Handgun,Yes,No,"No, No","No, No",https://www.orlando.gov/files/sharedassets/public/v/1/departments/public-safety/opd/open-data-docs/22-174307.pdf,POINT (-81.430522027 28.559823981)
2,18-448868,11/21/2018,9938 Universal Boulevard,Stolen Vehicle,1,"Chase, Alex",W,N,M,35,11,1,"Martinez Gonzalez, Joselito",White,M,30,Motor Vehicle,Yes,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-448869.pdf,POINT (-81.444407991 28.426330019)
3,18-219116,06/04/2018,4629 Cason Cove Drive,Attempted Suicide,1,"Sayer,Ryan",W,N,M,25,2,1,"White, Pamel",Black,F,63,Handgun,No,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-219116.pdf,POINT (-81.436934015 28.497201986)
4,20-397311,12/16/2020,"5434 Baldwin Park Street, Unit 101",Domestic Disturbance,1,missing,W,missing,M,40,14,1,"Robertson, Adam",White,M,34,Handgun,Yes,Yes,No,No,https://www.orlando.gov/files/sharedassets/public/departments/public-safety/opd/open-data-docs/ois_sao_zambito_ir-20-75.pdf,POINT (-81.31615297 28.574676001)
5,10-503738,10/26/2010,Hiawassee & SR 50,Armed Robbery,3,"Hinkles, Cedrick; Herb, Todd; Fischer, Bryan","B, W, W","N, N, N","M, M, M","30, 38, 37","6, 13, 4",1,"Soman, Acrifa",Black,M,21,Handgun,Yes,Yes,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/10-503738.pdf,POINT (-81.475974825 28.552001109)
6,13-022948,01/16/2013,1327 Arlington St,Drug Violation,2,"Mongeluzzo, Anthony; Bigelow, Chris","W, W","N, N","M, M","34, 29","6, 6",2,"Williams, Charles; Gamble, Karvas","Black, Black","M, M","37, 19",Handgun,"Yes, Yes","No, Yes","No, No","No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/13-022948.pdf,POINT (-81.398586976 28.552194982)
7,16-021812,01/16/2016,5400 North Lane,Traffic Stop,1,"Mills, Jonathan",W,N,M,33,4,1,"Lamons, Trevon",Black,M,17,Motor Vehicle,Yes,No,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/16-021812.pdf,POINT (-81.454123826 28.596308986)
8,14-086291,02/24/2014,595 W. Church St,Stalking,3,"Kipp, Alexander; Watts, Anthony; Johnston, David","W, B, W","N, N, N","M, M, M","32, 44, 24","1, 1, 1",1,"Lattimore, Derrick",Black,M,27,Motor Vehicle,No,No,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/14-086291.pdf,POINT (-81.386192991 28.540632985)
9,15-291159,07/17/2015,1657 Watauga Way,Suspicious Person,1,"Vidro, Kelvin",W,H,M,29,0.9,1,"Davis, Albert",Black,M,23,Hands/ASP,Yes,Yes,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/15-291159.pdf,POINT (-81.335092645 28.522822186)


# Graph Display and Analysis

In [4]:
# Function to convert Georeferenced Columm to Tuple 

function convert_to_tuple(str)
    test = str[7:end]
    test = strip(test, ['(', ')'])
    test = split(test, ' ')

    return (parse(Float64, test[1]), parse(Float64, test[2]))

end

convert_to_tuple (generic function with 1 method)

In [15]:
# Coorindate convertion
location = convert_to_tuple.(data[:, "Georeferenced Column"])

point = []

for each in location
    push!(point,LatLon(each[2],each[1]))
end

data[:, "Point"] = point

UndefVarError: UndefVarError: `LatLon` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
freq_by_type = combine(groupby(data, "Incident Type"), nrow => "Count")

Row,Incident Type,Count
,String,Int64
1,Vehicle Stop,1
2,Stolen Vehicle,4
3,Attempted Suicide,4
4,Domestic Disturbance,2
5,Armed Robbery,5
6,Drug Violation,2
7,Traffic Stop,7
8,Stalking,1
9,Suspicious Person,6


In [7]:
""" 
1. Group all incidents in categories: Shots, Suspicious person, vehicles, assault
2. Assign each category a color and/or icon type
3. Make Charts and Graphs with data
4. Create a Heat or Color map based on the markers
5. Maybe ad Police Stations on the map
6. Follow analysis with conclusions
"""

" \n1. Group all incidents in categories: Shots, Suspicious person, vehicles, assault\n2. Assign each category a color and/or icon type\n3. Make Charts and Graphs with data\n4. Create a Heat or Color map based on the markers\n5. Maybe ad Police Stations on the map\n6. Follow analysis with conclusions\n"

In [8]:

# Initialize map
m = foli.Map(location=[28.601, -81.429], zoom_start=12)

for i in 1:length(point)
    incident_type = data[i, "Incident Type"]  
    weapon = data[i, "Suspect's Weapon"]     
    case_number = data[i, "Case #"]          
    
    popup_text = """
    <b>Incident Type:</b> $incident_type<br>
    <b>Suspect's Weapon:</b> $weapon<br>
    <b>Case #:</b> $case_number<br>
    <b>Coordinates:</b> $(point[i].lat), $(point[i].lon)
    """
    
    foli.Marker(
        location=[point[i].lat, point[i].lon],  
        tooltip="Crime Location",
        popup=popup_text
    ).add_to(m)
end


m



PyObject <folium.folium.Map object at 0x144712620>

In [9]:
freq_by_type = combine(groupby(data, "Incident Type"), nrow => "Count")
sort!(freq_by_type, :Count, rev=true)

Row,Incident Type,Count
,String,Int64
1,Traffic Stop,7
2,Suspicious Person,6
3,Armed Robbery,5
4,Stolen Vehicle,4
5,Attempted Suicide,4
6,Armed Subject,4
7,Battery,3
8,Shots fired,3
9,Domestic Disturbance,2


In [10]:
# Colors and Icons
categories = Dict(
    "Shots" => ("red", "cloud"),
    "Suspicious Person" => ("blue", "user"),
    "Vehicles" => ("green", "car"),
    "Assault" => ("orange", "warning")
)

function categorize_incident(incident_type::String)
    for category in keys(categories)
        if occursin(category, incident_type)
            return categories[category]
        end
    end
    return ("gray", "info-sign")  
end

for i in 1:length(point)
    incident_type = data[i, "Incident Type"]  
    color, icon_type = categorize_incident(incident_type)
    
    foli.Marker(
        location=[point[i].lat, point[i].lon],
        tooltip=incident_type,
        popup="""
        <b>Incident Type:</b> $incident_type<br>
        <b>Coordinates:</b> $(point[i].lat), $(point[i].lon)
        """,
        icon=foli.Icon(color=color, icon=icon_type)
    ).add_to(m)
end
m

PyObject <folium.folium.Map object at 0x144712620>

In [11]:
using PyCall, Statistics

foli = pyimport("folium")
HeatMap = pyimport("folium.plugins").HeatMap

locations = [[each.lat, each.lon] for each in point]  

avg_lat = mean([loc[1] for loc in locations])
avg_lon = mean([loc[2] for loc in locations])
m = foli.Map(location=[avg_lat, avg_lon], zoom_start=12)

HeatMap(locations).add_to(m)

m


MethodError: MethodError: no method matching zero(::Type{Any})
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{LibGit2.GitHash})
   @ LibGit2 ~/.julia/juliaup/julia-1.11.1+0.x64.apple.darwin14/share/julia/stdlib/v1.11/LibGit2/src/oid.jl:221
  ...


In [12]:
# Police Stations in Orlando

police_stations = [
    (28.5402, -81.3807, "Orlando Police Headquarters"),  
    (28.5243, -81.4603, "District 2 Office")             
]

for (lat, lon, name) in police_stations
    foli.Marker(
        location=[lat, lon],
        tooltip="Police Station",
        popup="<b>$name</b>",
        icon=foli.Icon(color="darkblue", icon="shield", icon_size=(40, 40))
    ).add_to(m)
end
m



PyObject <folium.folium.Map object at 0x144712620>

# Conclusion

---

## Patterns Observed

- Incidents involving gunshots are predominantly concentrated in downtown Orlando, suggesting a link between high-activity zones and violent crimes.  
- Reports of suspicious persons are distributed across residential areas, raising concerns about community safety in these neighborhoods.  
- Vehicle-related incidents occur frequently in commercial zones, likely due to increased traffic and congestion in these areas.  
- Assaults show a mixed pattern, occurring both in central areas and certain residential pockets.  

---

## Insights

- The concentration of gunshots downtown highlights the need for focused **law enforcement presence** during peak hours and in high-traffic areas.  
- The distribution of suspicious person incidents indicates potential vulnerabilities in residential areas, which could benefit from community-based policing.  
- Frequent vehicle-related crimes in commercial zones emphasize the importance of patrolling parking lots and heavily trafficked streets.  

---

## Recommendations

1. Increase police presence in downtown Orlando during high-traffic periods to address violent crimes effectively.  
2. Establish neighborhood watch programs and deploy community officers to residential areas to enhance safety and foster trust.  
3. Implement targeted patrols in commercial zones to deter vehicle-related crimes and provide a safer environment for businesses and visitors.  

---

By focusing resources on identified crime hotspots and addressing specific patterns, law enforcement can take proactive measures to reduce criminal activities and enhance public safety across Orlando.

# CLARK CODE BEGINS 

# CLARK CODE ENDS

# UMA CODE BEGINS 

# UMA CODE ENDS 

## Conclusion



#### References: <br>
[OPD Officer-Involded Shootings](https://data.cityoforlando.net/Orlando-Police/OPD-Officer-Involved-Shootings/6kz6-6c7n/data_preview) <br>
[Folium Popups](https://python-visualization.github.io/folium/latest/user_guide/ui_elements/popups.html) <br>
[Folium Heatmap](https://python-visualization.github.io/folium/latest/user_guide/plugins/heatmap.html) <br>
[Folium Colormaps](https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html) <br>
[Folium Custon Icon](https://python-visualization.github.io/folium/latest/user_guide/ui_elements/icons.html) <br>